In [95]:
# '#'- for id
# '.'- for class

In [96]:
### Importing Library
from requests_html import HTMLSession
import pandas as pd

In [98]:
# Calling the library
s=HTMLSession()

# Making an empty list
gamelist=[]
gamelist2=[]

In [99]:
### Url of the site
#url= "https://store.steampowered.com/games/"
url= "https://store.steampowered.com/explore/new/"

In [100]:
#retrieving data from the url
re=s.get(url)

In [106]:
def parse(products):
    #for i in products:
        #m=i.absolute_links
        #print(m)
        #r=s.get(m)
    #for item in products.absolute_links:
        #print(i)
        #r=s.get(item)
    #for i in products:
        #print(i.absolute_links)
        #r=i.absolute_links
    
    for item in products.absolute_links:
        r=s.get(item)
        try:
            name=r.html.find('div.apphub_AppName',first=True).text
        except:
            continue
        
        #########Finding original price###############
        ##2
        try:
            original_price= r.html.find('div.discount_original_price',first=True).text.strip("₹,") 
        except:
            original_price= r.html.find('div.game_purchase_price',first=True).text.strip("₹,")
        
        #######finding discounted price if any########
    
        try:
            discounted_price= r.html.find('div.game_purchase_price',first=True).text.strip("₹,")
        except:
            discounted_price=r.html.find('div.discount_final_price',first=True).text.strip("₹,")
            
        ####################Finding top 5 tags#####################
        
        tag1=r.html.find('div.glance_tags.popular_tags a')[0].text
        
        try:
            tag2=r.html.find('div.glance_tags.popular_tags a')[1].text
        except:
            tag2=" "
        
        try:
            tag3=r.html.find('div.glance_tags.popular_tags a')[2].text
        except:
            tag3=" "
            
        try:
            tag4=r.html.find('div.glance_tags.popular_tags a')[3].text
        except:
            tag4=" "
            
        try:
            tag5=r.html.find('div.glance_tags.popular_tags a')[4].text
        except:
            tag5=" "
            
        #################Getting Number of Positive Reviews#############
        
        positive= r.html.find('span.responsive_hidden',first=True).text.strip("()") 
        
        ################# Finding developer Name#######################
        
        developer=r.html.find('div #developers_list',first=True).text
        
        ##################Finding Publisher name########################
        
        
        publisher=r.html.find('div.dev_row a')[1].text ## 1
        
        
        
        try:
            system=r.html.find('div.game_area_sys_req_leftCol',first=True).text
        except:
            system="None"
            
        ###############Finding App Id####################
        
        app_id=r.html.find('div.glance_tags.popular_tags',first=True).attrs['data-appid']
        
        
        game={
            'app_id':app_id,
            'name':name,
            'original_price':original_price,
            'discounted_price':discounted_price,
            'tag_1':tag1,
            'tag_2':tag2,
            'tag_3':tag3,
            'tag_4':tag4,
            'tag_5':tag5,
            'positive_reviews':positive,
            'developer':developer,
            'publisher':publisher,
            'system':system
            }
        gamelist.append(game)
        
        
        
        

##1 It is to note here that why does this code work here.. or why are we not usinh 'first=True'. It is because both developer and publisher uses same div, i.e. "dev_row". For learning purpose I used 'id'- to extract data out of developer.But in case of publisher there was no ID. We already know that 'html.find' creates a list, and that is why we uses 'first=True'. In dis case we did not want first data ([0]), we wanted second place ([1]). So we disabled 'first=True' and fetched data at [1] index. Try the following code for better understanding b=s.get("https://store.steampowered.com/app/911430/Good_Company/") print(b.html.find('div.dev_row a')[1].text)

##2 For prices there were two boxes. Box:1 - Which did not have discount and Box 2: Which had discount. So both the prices were extracted. It is to be noted that in the 'div' vales appeared like -- (₹12,00) i.e. Inside a bracked and with commas. While extracting, both were removes so that values were integer type.

In [136]:
# Giving xpath of the block from where game details need to be extracted


#p= re.html.find('div.tab_content a')[0].absolute_links #####Old link

p=re.html.xpath('//*[@id="tab_newreleases_content"]',first=True)


In [130]:
#Checking P to make sure it is the correct block
p

In [109]:
#Running the finction on the block
parse(p)

In [124]:
#### Converting gamelist to Dataframe####
gamelist=pd.DataFrame(gamelist)
gamelist.head()

,app_id,name,original_price,discounted_price,tag_1,tag_2,tag_3,tag_4,tag_5,positive_reviews,developer,publisher,system
0,1359980,POSTAL: Brain Damaged,529,498,Funny,Comedy,Memes,Action,FPS,"1,558","Hyperstrange, CreativeForge Games",CreativeForge Games,None
1,1949740,Banana Shooter,Free To Play,Free To Play,Action,Shooter,3D Platformer,3D,Multiplayer,646,CodingDaniel,CodingDaniel,Minimum:\n\nOS: Windows 7\n\nProcessor: Intel ...
2,747200,Keplerth,459,344,Adventure,Exploration,Sandbox,Sci-fi,Survival,"1,149",TARO,Gamera Games,Minimum:\n\nOS: Windows 7\n\nMemory: 2 GB RAM\...
3,1488200,Symphony of War: The Nephilim Saga,"2,005",529,RPG,Story Rich,Tactical RPG,Fantasy,Adventure,"1,492",Dancing Dragon Games,Freedom Games,Minimum:\n\nOS: Windows 10 / 98 / XP / Vista /...
4,1827980,Idle Spiral,Free To Play,Free To Play,Simulation,Idler,Casual,Clicker,Time Management,163,Archimedes Geeks,"Idle System, Inc.",None


In [116]:
for ids in gamelist['app_id']:
    
        
        r2=s.get(f'https://steamcommunity.com/app/'+ids+'/positivereviews/?browsefilter=toprated&snr=1_5_100010_&filterLanguage=all')
        
        #########Review 1#############
        try:
            data1= r2.html.find('div.apphub_CardTextContent')[0].text.split('\n')[1]
        except:
            data1="No review available"
            
        ######### Review 2 ###########

        try:
            data2= r2.html.find('div.apphub_CardTextContent')[1].text.split('\n')[1]
        except:
            data2="No review available"
            
        ######### Review 3 ###########
            
        try:
            data3= r2.html.find('div.apphub_CardTextContent')[2].text.split('\n')[1]
        except:
            data3="No review available"
            
        ########## Review 4 ##########
        
        try:
            data4= r2.html.find('div.apphub_CardTextContent')[3].text.split('\n')[1]
        except:
            data4="No review available"
        
        ######### Review 5 ###########
        
        try:
            data5= r2.html.find('div.apphub_CardTextContent')[4].text.split('\n')[1]
        except:
            data5="No review available"

            
        review={
            'app_id':ids,
            'review_1':data1,
            'review_2':data2,
            'review_3':data3,
            'review_4':data4,
            'review_5':data5
          
            }
        gamelist2.append(review)

In [120]:
#Converting gamelist to dataframe
gamelist2=pd.DataFrame(gamelist2)
gamelist2.head

,app_id,review_1,review_2,review_3,review_4,review_5
0,1359980,Like Psychonauts but instead of solving mental...,"The Circle of Life: First we had ""DOOM clones""...",Product received for free,"This can't be good for me, but I feel great","Мясо, кровь, стрельба, монстры, в этой игре не..."
1,1949740,add sex and maybe we will talk,今天是牛子节，点赞这段话，牛子精灵就会在你睡着的时候偷偷给你的牛子加长1cm，但是如果你看到...,this game is very,Banana,This is what Karlsons multiplayer is going to ...
2,747200,游戏买好了！没有朋友怎么办！,好玩，而且国产。,Product refunded,Early Access Review,Game looks like RimWorld but progresses like T...
3,1488200,"As a life-long fan of Fire Emblem, Langrisser,...","Terrible game. You can't romance Diana, the ho...",求中文！！！！！！！！！！,This feels like the spiritual successor to Ogr...,"I played this game while it was in beta, and k..."
4,1827980,You look at the spiral,⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⠀⣀⣀⡀⢀⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀,Junji Ito would approve,수학적 방정식의 변수들을 늘려주며 방정식대로 나타나는 나선을 점점 크게 키우는 방치...,The spirals are nice.
5,1118310,功能强大的整合模拟器，但是玩之前需要折腾，你需要在DLC页面下载游戏对应的核心，重启后加载核...,"""Why would I play this on steam?"" you may ask.",Yes! What a great idea with the SteamDeck just...,"Finally, RetroArch is officially released on S...","RetroArch é um laucher de emuladores, você ins..."
6,868270,选择入坑的朋友，耐心看完会对你有很大帮助！,刚进入游戏时一身绿装加满配步枪，加一把满配大狙被在背上，背包还又5万金，游戏5小时后，手里只...,完全ソロのエンジョイ紀行,"Никита Буянов, когда вайп?",我是傻逼
7,1359980,Like Psychonauts but instead of solving mental...,"The Circle of Life: First we had ""DOOM clones""...",Product received for free,"This can't be good for me, but I feel great","Мясо, кровь, стрельба, монстры, в этой игре не..."
8,1949740,add sex and maybe we will talk,今天是牛子节，点赞这段话，牛子精灵就会在你睡着的时候偷偷给你的牛子加长1cm，但是如果你看到...,this game is very,Banana,This is what Karlsons multiplayer is going to ...
9,747200,游戏买好了！没有朋友怎么办！,好玩，而且国产。,Product refunded,Early Access Review,Game looks like RimWorld but progresses like T...


In [121]:
final_gamelist=pd.merge(gamelist,gamelist2,left_on='app_id', right_on='app_id', how='inner')

In [123]:
#Lookig at gamelist 
final_gamelist.head()

,app_id,name,original_price,discounted_price,tag_1,tag_2,tag_3,tag_4,tag_5,positive_reviews,developer,publisher,system,review_1,review_2,review_3,review_4,review_5
0,1359980,POSTAL: Brain Damaged,529,498,Funny,Comedy,Memes,Action,FPS,"1,558","Hyperstrange, CreativeForge Games",CreativeForge Games,None,Like Psychonauts but instead of solving mental...,"The Circle of Life: First we had ""DOOM clones""...",Product received for free,"This can't be good for me, but I feel great","Мясо, кровь, стрельба, монстры, в этой игре не..."
1,1359980,POSTAL: Brain Damaged,529,498,Funny,Comedy,Memes,Action,FPS,"1,558","Hyperstrange, CreativeForge Games",CreativeForge Games,None,Like Psychonauts but instead of solving mental...,"The Circle of Life: First we had ""DOOM clones""...",Product received for free,"This can't be good for me, but I feel great","Мясо, кровь, стрельба, монстры, в этой игре не..."
2,1359980,POSTAL: Brain Damaged,529,498,Funny,Comedy,Memes,Action,FPS,"1,558","Hyperstrange, CreativeForge Games",CreativeForge Games,None,Like Psychonauts but instead of solving mental...,"The Circle of Life: First we had ""DOOM clones""...",Product received for free,"This can't be good for me, but I feel great","Мясо, кровь, стрельба, монстры, в этой игре не..."
3,1359980,POSTAL: Brain Damaged,529,498,Funny,Comedy,Memes,Action,FPS,"1,558","Hyperstrange, CreativeForge Games",CreativeForge Games,None,Like Psychonauts but instead of solving mental...,"The Circle of Life: First we had ""DOOM clones""...",Product received for free,"This can't be good for me, but I feel great","Мясо, кровь, стрельба, монстры, в этой игре не..."
4,1359980,POSTAL: Brain Damaged,529,498,Funny,Comedy,Memes,Action,FPS,"1,558","Hyperstrange, CreativeForge Games",CreativeForge Games,None,Like Psychonauts but instead of solving mental...,"The Circle of Life: First we had ""DOOM clones""...",Product received for free,"This can't be good for me, but I feel great","Мясо, кровь, стрельба, монстры, в этой игре не..."


In [94]:
#Converting final gamelist to csv
final_gamelist.to_csv('gamelist.csv',index=False)